In [1]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import numpy as np
import ast
from gensim.models import Word2Vec,KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
#df = pd.read_csv("../Data/Flujo1.csv", sep=";")
df = pd.read_csv('drive/My Drive/Tesis/Data/Flujo1.csv', sep=";")
df = df[df.Area == 'Social Externo']
df['Respuesta'] = df['Respuesta'].apply(ast.literal_eval)
list_raw_dimensions = df.Dimension.unique().tolist()
def enumerate_dimensions(dimension):
    return list_raw_dimensions.index(dimension)

df['Dimension'] = df.Dimension.apply(enumerate_dimensions)
model = Word2Vec.load('drive/My Drive/Tesis/modelWord2vec.bin')
list_responses = list(df.Respuesta)
list_dimensions = list(df.Dimension)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [3]:
def make_feature_vec(words, model, num_features):
    feature_vec = np.zeros((num_features,),dtype="float32")
    nwords = 0
    index2word_set = set(model.wv.index2word)
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1
            feature_vec = np.add(feature_vec,model[word])
    feature_vec = np.divide(feature_vec, nwords)
    return feature_vec
def get_avg_feature_vecs(reviews, model, num_features):
    counter = 0
    review_feature_vecs = np.zeros((len(reviews),num_features), dtype='float32')
    for review in reviews:
        review_feature_vecs[counter] = make_feature_vec(review, model, num_features)
        counter = counter + 1
    return review_feature_vecs

num_features = 10
list_responses_vec = get_avg_feature_vecs(list_responses, model, num_features)
x_train, x_test, y_train, y_test = train_test_split(list_responses_vec, list_dimensions, test_size = 0.2, random_state = 0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [4]:
lr = LogisticRegression(solver= 'liblinear', )
acc = cross_val_score(lr, list_responses_vec, list_dimensions, scoring='accuracy', cv=10)
print(acc)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linea

[0.41745758 0.4302359  0.43280576 0.39582734 0.3988198  0.40653802
 0.40567396 0.36075493 0.34884726 0.32925072]


In [5]:
lr.fit(x_train , y_train)
predicctions = lr.predict(x_test)
score = accuracy_score(y_test ,predicctions)
print("Predicting labels for test data..")
print(classification_report(y_test, predicctions))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Predicting labels for test data..
              precision    recall  f1-score   support

           0       0.41      0.22      0.29      1247
           1       0.30      0.19      0.23      1134
           2       0.43      0.04      0.08       645
           3       0.41      0.17      0.24       350
           4       0.42      0.57      0.48       988
           5       0.45      0.35      0.39       836
           6       0.58      0.30      0.40       432
           7       0.52      0.46      0.49       376
           8       0.57      0.50      0.53      1204
           9       0.43      0.20      0.27       403
          10       0.48      0.47      0.47      1257
          11       0.33      0.03      0.05      1341
          12       0.35      0.77      0.48      3292
          13       0.36      0.04      0.07       388

    accuracy                           0.40     13893
   macro avg       0.43      0.31      0.32     13893
weighted avg       0.41      0.40      0.36   

In [8]:
print(score)
print(confusion_matrix(y_test, predicctions))

0.4016411142301879
[[ 277   82    5    1   68   25   20    5   37    7   73   11  634    2]
 [  48  216    7    9  110   22   12   13   37   20   96   22  515    7]
 [  17   13   29    6  266   21    5   11   48    9   12    1  206    1]
 [  15    8    0   58   29   97   10   34    1    1    0    1   95    1]
 [  33   33    7    6  564   29    1   27   33    9   20    3  221    2]
 [  34    3    9   25   70  289   31   23   10    1    5    2  333    1]
 [  41    1    0   13   12   63  131    3    5    0    5    2  156    0]
 [   5    2    0    8   35   44    1  172    2    3    3    0  100    1]
 [  10   15    1    1   16    1    1    5  602    4   16    0  532    0]
 [   5   33    2    1   44    1    0   13   16   79   19    0  187    3]
 [  26   28    2    2   17    4    1    0    6    5  588    6  569    3]
 [  52   94    0    0   28   20    2    2   54    3  156   36  891    3]
 [ 104  105    5    8   56   28   11   14  188   16  213   16 2523    5]
 [   6   78    0    3   35    1 

In [7]:
probs = lr.predict_proba(x_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, probs)
auc = roc_auc_score(y_test, probs)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='AUC {:.3f}'.format(auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

ValueError: ignored